### **UJIAN AKHIR SEMESTER - SEMESTER GENAP 2023/2024**
#### **KECERDASAN BUATAN**
---------------------------------------------------------------------------
NAMA : MILKA PUTRI FIBIONA
#### NIM  : 71220851


In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error


# Load the dataset
file_path = '[Dataset]_Test_(Perawatan-Pesawat).csv'
data = pd.read_csv('[Dataset]_Test_(Perawatan-Pesawat).csv')

# Pengecekan Data Kotor
def check_data_quality(df):
    # Memeriksa nilai yang hilang
    missing_values = df.isnull().sum()

    # Memeriksa nilai duplikat
    duplicate_values = df.duplicated().sum()

    # Memeriksa nilai yang tidak valid (misalnya nilai yang sangat kecil atau sangat besar)
    outlier_values = {}
    for column in df.select_dtypes(include=[np.number]).columns:
        outliers = df[(df[column] < df[column].quantile(0.01)) | (df[column] > df[column].quantile(0.99))]
        outlier_values[column] = len(outliers)

    return missing_values, duplicate_values, outlier_values

# Memeriksa kualitas data
missing_values, duplicate_values, outlier_values = check_data_quality(data)
print("Missing Values:\n", missing_values)
print("\nDuplicate Values:\n", duplicate_values)
print("\nOutlier Values:\n", outlier_values)

# Pembersihan Data
def clean_data(df):
    # Mengisi nilai yang hilang dengan median atau cara lain yang sesuai
    df = df.fillna(df.median())

    # Menghapus nilai duplikat
    df = df.drop_duplicates()

    # Mengatasi nilai yang tidak valid dengan mengganti outlier menggunakan IQR
    for column in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
        df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])

    return df

# Membersihkan data
cleaned_data = clean_data(data)


Missing Values:
 id           0
cycle        0
setting1     0
setting2     0
setting3     0
            ..
sd20         0
sd21         0
ttf          0
label_bnc    0
label_mcc    0
Length: 71, dtype: int64

Duplicate Values:
 0

Outlier Values:
 {'id': 2, 'cycle': 2, 'setting1': 2, 'setting2': 0, 'setting3': 0, 's1': 0, 's2': 2, 's3': 2, 's4': 2, 's5': 0, 's6': 1, 's7': 2, 's8': 2, 's9': 2, 's10': 0, 's11': 1, 's12': 2, 's13': 2, 's14': 2, 's15': 2, 's16': 0, 's17': 1, 's18': 0, 's19': 0, 's20': 2, 's21': 2, 'av1': 0, 'av2': 2, 'av3': 2, 'av4': 2, 'av5': 0, 'av6': 0, 'av7': 2, 'av8': 2, 'av9': 2, 'av10': 0, 'av11': 2, 'av12': 2, 'av13': 2, 'av14': 2, 'av15': 2, 'av16': 0, 'av17': 2, 'av18': 0, 'av19': 0, 'av20': 2, 'av21': 2, 'sd1': 0, 'sd2': 2, 'sd3': 2, 'sd4': 2, 'sd5': 0, 'sd6': 1, 'sd7': 2, 'sd8': 2, 'sd9': 2, 'sd10': 0, 'sd11': 2, 'sd12': 2, 'sd13': 2, 'sd14': 2, 'sd15': 2, 'sd16': 0, 'sd17': 1, 'sd18': 0, 'sd19': 0, 'sd20': 2, 'sd21': 2, 'ttf': 2, 'label_bnc': 0, 'label_mcc': 0}

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 71 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         100 non-null    int64  
 1   cycle      100 non-null    int64  
 2   setting1   100 non-null    float64
 3   setting2   100 non-null    float64
 4   setting3   100 non-null    float64
 5   s1         100 non-null    float64
 6   s2         100 non-null    float64
 7   s3         100 non-null    float64
 8   s4         100 non-null    float64
 9   s5         100 non-null    float64
 10  s6         100 non-null    float64
 11  s7         100 non-null    float64
 12  s8         100 non-null    float64
 13  s9         100 non-null    float64
 14  s10        100 non-null    float64
 15  s11        100 non-null    float64
 16  s12        100 non-null    float64
 17  s13        100 non-null    float64
 18  s14        100 non-null    float64
 19  s15        100 non-null    float64
 20  s16        

In [47]:
data.duplicated().sum()

0

In [48]:
# Normalisasi Data
def normalize_data(df):
    # Drop irrelevant columns for normalization
    columns_to_normalize = df.drop(columns=['id', 'cycle', 'label_bnc', 'label_mcc', 'ttf']).columns

    # Initialize the MinMaxScaler
    scaler = MinMaxScaler()

    # Fit and transform the data
    df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

    return df

# Normalisasi data yang telah dibersihkan
normalized_data = normalize_data(cleaned_data)

# Drop irrelevant columns
normalized_data = normalized_data.drop(columns=['id', 'cycle', 'label_bnc', 'label_mcc'])

# Memastikan hanya kolom yang relevan yang digunakan untuk prediksi
X = normalized_data.drop(columns=['ttf'])
y = normalized_data['ttf']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat Model Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediksi
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# Evaluasi Model
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Train MSE: {train_mse}, Test MSE: {test_mse}")
print(f"Train R2: {train_r2}, Test R2: {test_r2}")

# Fungsi untuk Mengonversi Prediksi ttf ke Persentase Kerusakan
def predict_damage_percentage(features, model, max_ttf=200):
    ttf_pred = model.predict(features)
    ttf_pred = np.clip(ttf_pred, 0, max_ttf)  # Pastikan nilai prediksi tidak melebihi max_ttf
    damage_percentage = 100 * (max_ttf - ttf_pred) / max_ttf
    return damage_percentage

# Contoh penggunaan dengan satu sampel dari X_test
sample_features = X_test.iloc[0].values.reshape(1, -1)
damage_percentage = predict_damage_percentage(sample_features, model)
print(f"Predicted Damage Percentage: {damage_percentage[0]:.2f}%")

Train MSE: 75.71281875, Test MSE: 381.33827999999994
Train R2: 0.9568274241995096, Test R2: 0.7617718943417915
Predicted Damage Percentage: 71.76%


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [49]:
# Evaluasi Model dengan MAE
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"Train MAE: {train_mae}, Test MAE: {test_mae}")

# Evaluasi Model dengan R-squared
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print(f"Train R2: {train_r2}, Test R2: {test_r2}")


Train MAE: 6.2731249999999985, Test MAE: 13.798000000000002
Train R2: 0.9568274241995096, Test R2: 0.7617718943417915


In [50]:
# Presentasi akurasi
# Evaluasi Model dengan MAE
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)

print(f"Train MAE: {train_mae}, Test MAE: {test_mae}")

# Akurasi prediksi persentase kerusakan
accuracy = 100 - test_mae  # Karena persentase kerusakan dihitung dari 100 - MAE
print(f"Accuracy of Damage Percentage Prediction: {accuracy:.2f}%")


Train MAE: 6.2731249999999985, Test MAE: 13.798000000000002
Accuracy of Damage Percentage Prediction: 86.20%
